# Tensorflow 

In [1]:
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

### 创造第一个图谱，然后运行它

可以看出 tf 不进行任何计算，即使在最后一行中的 f 定义了计算关系，__实际上他只是创建一个计算图谱，任何变量都未初始化__。

__要求出此图，你需要建立一个新的会话，并使用它初始化变量并且求出f__。tf 会话负责处理在诸如 CPU 和 GPU 的设备上运行，并保留变量值。

In [3]:
import tensorflow as tf
x = tf.Variable(3,name='x')
y = tf.Variable(4,name='y')
f = x*x*y+y+2
f

<tf.Tensor 'add_3:0' shape=() dtype=int32>

### 创建会话，结束后删除会话

In [4]:
# way1
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

display(result)
sess.close()

42

在	__with__	块中，会话被设置为 __默认会话__。 

调用	__x.initializer.run()__	等效于调用	__tf.get_default_session().run(x.initial)__	

__f.eval()__	等效于调用	__tf.get_default_session().run(f)__	

In [6]:
# way2
with tf.Session() as sess:
    
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

    display(result)


42

你可以使用	__global_variables_initializer()__		函数，而不是手动初始化每个变量。	

请注意，__它实际上没有立即执行初始化，而是在图谱中创建一个当程序运行时所有变量都会初始化的节点__：


In [7]:
# way3
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()             # 初始化
    result = f.eval()
    display(result)

42

在	__Jupyter__	内部或在	__Python	shell__	中，您可能更喜欢创建一个	__InteractiveSession__	。	

与常规会话的唯一区别是，当创建	__InteractiveSession__	时，它将自动将其自身设置为默认会话，因此您不需要使用模块（但是您需要在完成后手动关闭会话）：


In [8]:
# way4 
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
display(result)

42

### 管理图谱

__值得注意的是：__ 你创建的任何节点都会自动添加到 `默认图形` 中

In [9]:
x = tf.Variable(1)
x.graph is tf.get_default_graph()

True

在大多数情况下，这是很好的，但有时您可能需要 `管理多个独立图形`。	

您可以通过 __创建一个新的图形并暂时将其设置为一个块中的默认图形__ ，如下所示：


In [11]:
graph2 = tf.Graph()
with graph2.as_default():
    x2 = tf.Variable(2)
display(x2.graph is graph2)

x2.graph is tf.get_default_graph()

True

False

在	__Jupyter（或	Python	shell）__ 中，通常在实验时多次运行相同的命令。	因此，您可能会收到包含许多重复节点的默认图形。	

一个解决方案是 `重新启动`	__Jupyter	内核（或	Python	shell）__， 但是一个更方便的解决方案是通过运行	`tf.reset_default_graph()`	来 __重置默认图__。


### 节点的生命周期


#### tensorflow 数据流图 

<div><img width="400" height="300" src="static/数据流图1.jpg"/><img width="400" height="300" src="static/数据流图2.jpg"/></div>


#### 节点：

通过模型训练的视角可以将节点分为三类，分别是计算节点 存储节点和数据节点

`计算节点(operation)`：

	用于计算的一些操作包括一些逻辑的操作，神经网络的操作，一些简单的加法的操作和一些规约操作等等 (我们后面会介绍)
    
`存储节点(variable)`：假设模型为: s=wx+b 

    其实就是我们的variable就是我们的变量在图中我们可以看出我们的 w (权重) 和 b (偏置) 模型的权重和偏置,那模型的权重和偏置就是我们数据流图或者说是神经网络的训练的对象，我们其实上就是为了实现一个能解决我们实际上问题的w和b，也就是像找到一个能解决出我们问题的函数，w和b就非常重要，其实我们的逻辑运算，我们的神经网络层，我们的一些矩阵相乘这些逻辑其实都是不变的，但是我们输入的数据会变化，我们每一次训练都会丢入新的数据来训练，那这个时候，我们的w和b就是会不断的迭代更新的，我们的variable其实就是用来存放我们迭代的模型参数的。

`数据节点(placeholder)`：

    也就是我们图里面的input，input其实包含class labels(分类模型里面的标签) 在监督学习里面我们通常会使用这种方式。

    这时候我们其实是需要输入新的数据到数据集里面的，他们不是这个图里面的计算逻辑，而是说我们的训练数据，我们的测试数据，其实这些数据也是需要我们去描述和表达的，Placeholder其实就是用来表示图外输入的数据，它回去描述数据的类型，数据的形状，当这个图描述完成之后，还没有真正开始进行运算，真正开始运算其实是从input节点开始的，当我们真正把数据输入到数据流图之后才会真正的开始训练，当图走完之后，我们通过损失值算出了梯度，通过梯度下降我们求出了我们优化的值是多少然后将我们优化出的值更新我们的这一轮的模型参数，然后一轮的模型也就更新完成。


求出节点时，__TensorFlow__	会自动确定 __所依赖的节点集__ ，并 __首先__ 求出这些节点。

In [18]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x + 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())    

10
8


上面这段代码定义了一个简单的 `graph` ，并计算 `y` 和 `z` 的值，`TF` 发现 `y` 依赖 `x` 、`x` 依赖 `w` 。所以它依次计算 `w`、`x`和`y`。在计算`z`的时候，它 __不会复用之前的计算结果__ ，会再次计算 `x` 和 `w`。__最终这段代码执行了两次w和x__。

__所有节点值都在图运行之间删除，除了变量值，除了Variable值__ ，`variable` 的生命周期为整个 `session` 。也就是说 `variable` 的生命周期从 `initializer` 开始，到 `session close` 结束。

上面这段代码在正式的生产环境下效率是很低的，为了避免被重复计算，我们就需要告诉TF计算 `y` 和 `z` 在同一个 `graph` 中。下面是代码：

In [17]:
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)    

10
8


__注意：__ 在单进程的TF程序中，多个session是不共用变量(数据)的，每一个session有着独自的变量copy。在分布式TF程序中，变量是存储在server，而不是在session中，所以多个session可以共享变量。

## Linear	Regression	with	TensorFlow


在之前我们使用 __`sklean进行数据挖掘`__ ，这里我们使用TF来进行计算，不过为了方便我们直接使用sklean提供的数据集，跳过数据处理过程,

直接使用 __`正规方程(Normal Equation)方法`__ 求解: $$θ=(X^T⋅X)^{−1}⋅ X^T⋅y$$

类似 __Numpy__ ，TF也提供了许多数据转换的方法，在numpy数组被成为ndarray，详见掌握numpy，__在TF中的多维数组被成为张量(tensors)__。

In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [5]:
housing =  fetch_california_housing()
m,n = housing.data.shape
#np.c_按colunm来组合array
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data] # 在所有的数据前面再加上一个新的特征，特征值为1
x = tf.constant(housing_data_plus_bias,dtype=tf.float32,name='x')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
xT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xT,x)),xT),y)

with tf.Session() as sess:
    theta_values = theta.eval()
    print(theta_values)

[[-3.7171074e+01]
 [ 4.3633682e-01]
 [ 9.3871783e-03]
 [-1.0717344e-01]
 [ 6.4540231e-01]
 [-4.1238391e-06]
 [-3.7809242e-03]
 [-4.2373490e-01]
 [-4.3720812e-01]]


上面这段代码可以完全使用Numpy替代，当然也可以使用sklearn的回归方法，也是分分钟搞定的事情

In [39]:
from sklearn.linear_model import LinearRegression

In [40]:
line_model = LinearRegression()
line_model.fit(housing.data,houring.target.reshape(-1,1))
np.vstack((line_model.intercept_.reshape(-1,1),line_model.coef_.T))   # intercept_ 为截距 coef_ 为系数

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

### 实现梯度下降

__注意：__ 当使用梯度下降时，请记住，__首先要对输入特征向量进行归一化__，否则训练可能要慢得多。比如可以使用 skleran 中的 standerScaler

<div><img width='400',hight='500' src="static/梯度下降.png"/></div>



#### tf.random_uniform:

    tf.random_uniform([rows, colomns], maxval = high, minval = low, dtype = tf.float32)

    返回一个维度为[rows, colomns]，范围为[low, high]的均匀分布随机浮点数张量

#### tf.reduce_mean:

    import tensorflow as tf
    sess = tf.InteractiveSession()

    x = [[1., 2.], [3., 4.]]

    mean1 = tf.reduce_mean(x) # 如果不指定第二个参数，那么就在所有的元素中取平均值 2.5
    print(sess.run(mean1))

    mean2 = tf.reduce_mean(x, 0) # 指定第二个参数为0，则第一维的元素取平均值，即每一列求平均值   [2. 3.]
    print(sess.run(mean2))

    mean3 = tf.reduce_mean(x, 1) # 指定第二个参数为1，则第二维的元素取平均值，即每一行求平均值   [1.5 3.5]
    print(sess.run(mean3)) 
    
#### tf.assign 

    tf.assign(ref, value, validate_shape=None, use_locking=None, name=None)

    函数完成了将value赋值给ref的作用。其中：ref 必须是tf.Variable创建的tensor，如果ref=tf.constant()会报错！

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [27]:
housing = fetch_california_housing()
m,n = housing.data.shape
# np.c_按colunm来组合array
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]
scaler_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.1

X = tf.constant(scaler_housing_data_plus_bias,dtype=tf.float32,name='X')
Y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='Y')

theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')

error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')

gradients = 2/m * tf.matmul(tf.transpose(X),error)    # 梯度向量
training_op = tf.assign(theta,theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("Epoch: ",epoch,"MSE: ",mse.eval())
            sess.run(training_op)
    best_theta = theta.eval()

Epoch:  0 MSE:  7.5397058
Epoch:  100 MSE:  6.6102448
Epoch:  200 MSE:  6.0585012
Epoch:  300 MSE:  5.7167296
Epoch:  400 MSE:  5.4982347
Epoch:  500 MSE:  5.355001
Epoch:  600 MSE:  5.2589726
Epoch:  700 MSE:  5.1931186
Epoch:  800 MSE:  5.1468325
Epoch:  900 MSE:  5.113388


### TF 自动计算梯度

上面代码通过手动计算损失函数导数的迭代公式计算出θ的值，一个线性回归手动算起来固然容易，但当模型为一个神经网络再进行手动求导就会很吃力了。

TF提供了自动求导功能，__只需用以下面这行代码替换上一节中代码的	gradients	=	...	行，代码将继续工作正常。__

In [29]:
gradients = tf.gradients(mse,[theta])[0]

`gradients() 函数` 使用一个 op （在这种情况下是MSE）和一个变量列表（在这种情况下只是 theta ），它创建一个	ops	列表（每个变量一个）来计算 op 的梯度变量。	

因此，__梯度节点将计算 MSE 相对于 theta 的梯度向量__。